<a href="https://colab.research.google.com/github/emilioiturbide/COSC-419-Lab-5/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5

In [1]:
pip install torch transformers datasets peft scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
dataset = load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

In [7]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [8]:
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=16,
    logging_steps=500,
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [13]:
eval_result = trainer.evaluate()
print("Pre-trained Model Evaluation:", eval_result)
pre_trained_eval_result = eval_result

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Pre-trained Model Evaluation: {'eval_loss': 0.7179046869277954, 'eval_model_preparation_time': 0.0063, 'eval_accuracy': 0.50048, 'eval_f1': 0.01916431039899466, 'eval_precision': 0.5258620689655172, 'eval_recall': 0.00976, 'eval_runtime': 196.9608, 'eval_samples_per_second': 126.929, 'eval_steps_per_second': 7.936}


In [14]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence Classification
    inference_mode=False,
    r=8,  # Low-rank dimension
    lora_alpha=32,
    lora_dropout=0.1,
)

In [15]:
model = get_peft_model(model, peft_config)

In [16]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # For demonstration; adjust as needed
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.367500,0.346160,0.848960,0.850775,0.840675,0.861120


TrainOutput(global_step=1563, training_loss=0.433827602169259, metrics={'train_runtime': 643.7907, 'train_samples_per_second': 38.832, 'train_steps_per_second': 2.428, 'total_flos': 1650135936000000.0, 'train_loss': 0.433827602169259, 'epoch': 1.0})

In [19]:
eval_result = trainer.evaluate()
print("Fine-Tuned Model Evaluation:", eval_result)
fine_tuned_eval_result = eval_result

Fine-Tuned Model Evaluation: {'eval_loss': 0.34615960717201233, 'eval_accuracy': 0.84896, 'eval_f1': 0.8507745810938983, 'eval_precision': 0.8406747891283973, 'eval_recall': 0.86112, 'eval_runtime': 211.7934, 'eval_samples_per_second': 118.04, 'eval_steps_per_second': 7.38, 'epoch': 1.0}


* Compare the Results before and after fine-tuning

```
•	Accuracy
•	F1 Score
•	Precision
•	Recall
```


```
Pre-trained Model Evaluation: {'eval_loss': 0.6931, 'eval_accuracy': 0.5, ...}
Fine-Tuned Model Evaluation: {'eval_loss': 0.2456, 'eval_accuracy': 0.90, ...}
```

In [20]:
print("Pre-trained Model Evaluation: ", pre_trained_eval_result)
print("Fine-Tuned Model Evaluation: ", fine_tuned_eval_result)

Pre-trained Model Evaluation:  {'eval_loss': 0.7179046869277954, 'eval_model_preparation_time': 0.0063, 'eval_accuracy': 0.50048, 'eval_f1': 0.01916431039899466, 'eval_precision': 0.5258620689655172, 'eval_recall': 0.00976, 'eval_runtime': 196.9608, 'eval_samples_per_second': 126.929, 'eval_steps_per_second': 7.936}
Fine-Tuned Model Evaluation:  {'eval_loss': 0.34615960717201233, 'eval_accuracy': 0.84896, 'eval_f1': 0.8507745810938983, 'eval_precision': 0.8406747891283973, 'eval_recall': 0.86112, 'eval_runtime': 211.7934, 'eval_samples_per_second': 118.04, 'eval_steps_per_second': 7.38, 'epoch': 1.0}
